In [18]:
import pandas as pd
import nltk
import string
import re
import h2o
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html

In [3]:
tmp_df = pd.read_csv('D:/marco/vr_chat-master/Marco/train_yn.csv')
tmp_df.head()

,passages,query,query_id,query_type,answers_yn
0,"[Row(is_selected=0, passage_text=u'In his youn...",was ronald reagan a democrat,19700,description,Yes
1,"[Row(is_selected=0, passage_text=u'So, in the ...",do game shows pay their contestants,19749,description,Yes
2,"[Row(is_selected=0, passage_text=u'Even though...",does peanut butter contain cholesterol,19761,description,No
3,"[Row(is_selected=0, passage_text=u'When turnin...",can you take left hand lane to turn right on a...,19762,description,No
4,"[Row(is_selected=0, passage_text=u'Assault on ...",is armed assault a felony,19774,description,Yes


In [42]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

from nltk.stem.porter import *


In [72]:
# def tf_cln(sent = tmp_df.iloc[0][0],cln=True):
cln=True
sent = tmp_df.iloc[0][0]

lowers= sent.lower()
# print lower
no_punctuation = lowers.translate(None, string.punctuation)


slt_sent = re.findall(r'passagetext(.*?)url',no_punctuation)
# print slt_sent 

# if(cln==True):
#     slt_sent = re.findall(r'passagetext(.*?)url',no_punctuation)
# #         print slt_sent
# else:
#     slt_sent = sent
    
token = nltk.word_tokenize(str(slt_sent))
filtered_sentence = [w for w in token if not w in stop_words]

for i in filtered_sentence:
    if re.findall(r'^\W.*',str(i)):
        stop_words.add(i)

stop_words.add('``')
filtered_sentence = [w for w in token if not w in stop_words]

# stemmer = PorterStemmer()
# stm_lst = []
# for i in filtered_sentence:
#     stm_lst.append(stemmer.stem(i))

# print filtered_sentence

tf = nltk.FreqDist(filtered_sentence)
# print dict(tf)

tfidf = TfidfVectorizer(tokenizer=filtered_sentence, stop_words=stop_words)

# TfidfVectorizer
# tfidf.fit_transform
# tfs = tfidf.fit_transform(no_punctuation)

#     return dict(tf)

In [77]:
nltk.FreqDist(tfidf.tokenizer)

FreqDist({'1911': 1,
          '19112004': 1,
          '1935': 2,
          '1948': 1,
          '1960s': 2,
          '1962': 2,
          '1967': 1,
          '1970s': 1,
          '1975': 1,
          '1980': 2,
          '1981': 2,
          '1984': 4,
          '1989': 2,
          '2004': 1,
          '20s': 1,
          '40th': 2,
          '49': 1,
          '5': 1,
          '50': 1,
          '51': 1,
          '525': 1,
          '538': 1,
          '6': 1,
          '8': 1,
          'actor': 3,
          'age': 1,
          'america': 1,
          'american': 2,
          'audio': 1,
          'became': 3,
          'began': 1,
          'bill': 1,
          'california': 3,
          'campaigned': 2,
          'candidate': 3,
          'candidates': 1,
          'carried': 1,
          'carter': 1,
          'challenger': 1,
          'changed': 2,
          'clinton': 1,
          'commentator': 1,
          'conservative': 2,
          'country': 1,
          'defeatin

In [ ]:
tfs = tfidf.fit_transform

In [16]:
dic = tf_cln()